<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/my_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install youtokentome

In [11]:
import math
import numpy as np
import pandas as pd
import json
import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [6]:
device = torch.device('cuda:0')

In [15]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [17]:
!unzip '/content/drive/My Drive/unsupervised.csv.zip' 

Archive:  /content/drive/My Drive/unsupervised.csv.zip
caution: filename not matched:  /content/drive/qa_data.jsonl.zip


In [19]:
!unzip '/content/drive/My Drive/qa_data.jsonl.zip'

Archive:  /content/drive/My Drive/qa_data.jsonl.zip
  inflating: qa_data.jsonl           
  inflating: __MACOSX/._qa_data.jsonl  


In [8]:
# чтобы прочитать весь файл

qa_data = list()

with open('qa_data.jsonl') as file_object:
    for line in file_object:
        qa_data.append(json.loads(line.strip()))
file_object.close()

In [9]:
qa_data[:3]

[{'category': 'Бизнес, Финансы',
  'question': 'долго ли идут деньги с яндексденег на карту visa?',
  'responses': ['нет. прорыв 35 ;)']},
 {'category': 'Авто, Мото',
  'question': 'можно ли зарегистрировать авто в другом регионе',
  'responses': ['можно на родственника из того региона.. .  а потом ездить по доверке']},
 {'category': 'Красота и Здоровье',
  'question': 'что делать если у меня очень тонкие ногти а хочется их отрастить?',
  'responses': ['витамины и умная эмаль (каждый день)',
   'ванночки с морской солью. с вечера мажь ногти сверху йодом. не бойся, до утра все впитается.',
   'умная эмаль, витамины, йод, и поменьше крась лаком ',
   'лаки фирмы trind производство usa + кальций']}]

In [19]:
my_df = pd.DataFrame(qa_data)
my_df

,question,category,responses
0,долго ли идут деньги с яндексденег на карту visa?,"Бизнес, Финансы",[нет. прорыв 35 ;)]
1,можно ли зарегистрировать авто в другом регионе,"Авто, Мото",[можно на родственника из того региона.. . а ...
2,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,"[витамины и умная эмаль (каждый день), ванночк..."
3,в чем отличие медитации от йоги?,Спорт,"[букв в йоге меньше, в медитации ты просто сид..."
4,когда начнут линейку фильмов про лигу чемпионов?,Спорт,[а не фильм? жалко... а я то думал - хорошая к...
...,...,...,...
2808806,какой у вас обхват запястья в сантиметрах? в о...,Красота и Здоровье,[19 см м 92 20 лет]
2808807,завидуете ли вы удачам и достижениям знакомых ...,"Знакомства, Любовь, Отношения","[я очень рада за них., нет никогда! в основном..."
2808808,есть ли такая машина land rover range rover sp...,"Авто, Мото",[это две разные машины.]
2808809,"во что инвестировать, с чем работать, идеи?","Бизнес, Финансы",[вешенки можно выращивать на пнях. нормальная ...


In [21]:
with open('for_bpe.txt', 'w', encoding='utf-8') as f:
    for que in my_df['question']:
        f.write(que + '\n')
f.close()

vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'

In [22]:
%%time
# обучаем
# раскомментируйте этот код, чтобы обучить bpe
yttm.BPE.train(data='for_bpe.txt', vocab_size=vocab_size, model=model_path)

CPU times: user 22.7 s, sys: 2.01 s, total: 24.7 s
Wall time: 14.7 s


In [23]:
tokenizer = yttm.BPE(model=model_path)

In [7]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [10]:
questions

,question
0,"положено ли пособие опекаемому ребенку, обучаю..."
1,где скачать прогу для записи видео онлайн. зар...
2,физика амперы метрология помогите
3,"имеет ли многодетная семья ""скидку"" на услуги ..."
4,пенсия по инвалидности и старости...
...,...
2535438,подскажите серию мультфильма - кураж трусливый...
2535439,что делать если без опыта берут только на рабс...
2535440,как определить жанр книги?
2535441,шрифт see regular что-то имитирует? или просто...
